In [23]:
(define pi 3.14)

In [24]:
(define radius 10)

In [25]:
(* pi (* radius radius))

314.0

### 1.1.7 例: ニュートン法による平方根

In [26]:
(define (sqrt-iter guess x)
  (if (good-enough? guess x)
      guess
      (sqrt-iter (improve guess x) x)))

In [27]:
(define (improve guess x)
  (average guess (/ x guess)))

In [28]:
(define (average x y) (/ (+ x y) 2))

In [29]:
(define (good-enough? guess x)
  (< (abs (- (square guess) x)) 0.001))

In [30]:
(define (square x) (* x x))

In [31]:
(define (srqt x)
  (sqrt-iter 1.0 x))

In [32]:
(srqt 2)

1.4142156862745097

In [33]:
(sqrt 3)

1.7320508075688772

In [34]:
(sqrt 9)

3.0

## 練習問題 1.6

In [35]:
(define (new-if predicate then-clause else-clause)
  (cond (predicate then-clause)
        (else else-clause)))

In [36]:
(define (sqrt-iter-mod guess x)
  (new-if (good-enough? guess x)
      guess
      (sqrt-iter-mod (improve guess x) x)))
(define (srqt-mod x)
  (sqrt-iter-mod 1.0 x))

In [37]:
// This cause infinite loop
// (srqt-mod 3)


Traceback (most recent call last):
  File "In [37]", line 1, col 4
RunTimeError: unbound variable 'This'



`(new-if (good-enough? guess x) guess (sqrt-iter-mod (improve guess x) x))` を呼び出す時点で `else-clause` の値を算出するために `(sqrt-iter-mod (improve guess x) x)` が評価される。そのため呼び出しごとに必ず再起処理が発生して無限に呼び出しスタックが深くなる。よって計算が終了しない。

## 練習問題 1.7

## 練習問題 1.8

In [38]:
(define (third-root-iter guess x)
  (if (good-enough? guess x)
      guess
      (third-root-iter (improve-tr guess x) x)))

In [39]:
(define (improve-tr guess x)
  (display (format "x: ~a, guess: ~a\n" guess x))
  (/ 
    (+ 
      (/ x (square guess)) 
      (* guess 2))
    3))

In [40]:
(define (third-root x)
  (third-root-iter 1.0 x))

In [41]:
(define (third-power x) (* (* x x) x))

(define (good-enough? guess x)
  (< (abs (- (third-power guess) x)) 0.001))

In [42]:
(third-root 27.0)

x: 1.0, guess: 27.0
x: 9.666666666666666, guess: 27.0
x: 6.540758356453956, guess: 27.0
x: 4.570876778578707, guess: 27.0
x: 3.4780192333867963, guess: 27.0
x: 3.0626891086275365, guess: 27.0
x: 3.001274406506175, guess: 27.0


3.0000005410641766

## 1.2 手続きとそれが生成するプロセス

In [43]:
(define (factorial n)
  (if (= n 1)
    n
    (* n (factorial (- n 1)))))

In [45]:
(factorial 6)

720

この定義の場合、再起呼び出しはネストしていく
```s
(factorial 6)
(* 6 (factorial 5))
...
(* 6 (* 5 (* 4 (* 3 (* 2 (factorial 1))))))
(* 6 (* 5 (* 4 (* 3 (* 2 1)))))
...
(* 6 120)
720
```

線型なプロセスで計算できるようにする
```s
(fact-iter acc counter max)
```
という関数を定義すると以下のようになる

```s
(factorial 6)
(fact-iter 1 1 6)
(fact-iter 1 2 6)
(fact-iter 2 3 6)
(fact-iter 6 4 6)
...
(fact-iter 720 7 6)
```

In [49]:
(define (factorial n)
  ;;; n is max of counte. as it won't change, we don't need to pass to iter
  (define (iter product counter)
    (if (> counter n)
      product
      (iter (* counter product)
            (+ counter 1))))
  (iter 1 1)
)

In [50]:
(factorial 6)

720

- recursive process
  - first example
  - build up a chain of deferred operations, then contraction occurs
  - linear recursive process
    - chain grows linearly with N
- iterative process
  - second example
  - process does not grow and shrink
  - state is managed by state variables
  - linear iterative process
    - number of steps grows linearly with N

Distinguish `recuresive process` and `recureive procedure`.

- recuresive procedure: systematic fact that the procedure definition referes to the procedure itself.
- recuresive process: how the process evolves.

## Exercise 1.9:
Each of the following two procedures defines a method for adding two positive integers in terms of the procedures inc, which increments its argument by 1, and dec, which decrements its argument by 1.
```s
(define (+ a b)
  (if (= a 0) 
    b 
    (inc (+ (dec a) b))))
(define (+ a b)
  (if (= a 0) 
    b 
    (+ (dec a) (inc b))))
```
Using the substitution model, illustrate the process generated by each procedure in evaluating (+ 4 5). Are these processes iterative or recursive?

In [2]:
(define (dec a) (- a 1))
(define (inc a) (+ a 1))
(define (sum a b)
  (if (= a 0) 
    b 
    (inc (sum (dec a) b))))

In [4]:
(sum 4 5)

9

First process
```s
(+ 4 5)
(inc (+ (dec 4) 5)) -> (inc (+ 3 5))
(inc (inc (+ (dec 3) 5))) -> (inc (inc (+ 2 5)))
(inc (inc (inc (+ (dec 2) 5)))) -> (inc (inc (inc (+ 1 5))))
(inc (inc (inc (inc (+ (dec 1) 5))))) -> (inc (inc (inc (inc (+ 0 5)))))
(inc (inc (inc (inc (+ 0 5)))))
(inc (inc (inc (inc 5))))
(inc (inc (inc 6)))
(inc (inc 7))
(inc 8)
9
```
-> recursive process

Second process
```s
(+ 4 5) -> (+ (dec 4) (inc 5))
(+ 3 6) -> (+ (dec 3) (inc 6))
(+ 2 7) -> (+ (dec 2) (inc 7))
(+ 1 8) -> (+ (dec 1) (inc 8))
(+ 0 9)
9
```
-> iterative process

## Exercise 1.10:
Thee following procedure computes a mathematical function called Ackermann’s function.
```s
(define (A x y)
  (cond ((= y 0) 0)
        ((= x 0) (* 2 y))
        ((= y 1) 2)
        (else (A (- x 1) (A x (- y 1))))))
```
What are the values of the following expressions?
```s
(A 1 10)
(A 2 4)
(A 3 3)
```
Consider the following procedures, where A is the procedure defined above:
```s
(define (f n) (A 0 n))
(define (g n) (A 1 n))
(define (h n) (A 2 n))
(define (k n) (* 5 n n))
```
Give concise mathematical definitions for the functions computed by the procedures f, g, and h for positive integer values of n. For example, (k n) computes 5n.


In [2]:
(define (A x y)
  (display (format "x: ~a, y: ~a\n" x y))
  (cond ((= y 0) 0)
        ((= x 0) (* 2 y))
        ((= y 1) 2)
        (else (A (- x 1) (A x (- y 1))))))

In [6]:
(A 1 10)

x: 1, y: 10
x: 1, y: 9
x: 1, y: 8
x: 1, y: 7
x: 1, y: 6
x: 1, y: 5
x: 1, y: 4
x: 1, y: 3
x: 1, y: 2
x: 1, y: 1
x: 0, y: 2
x: 0, y: 4
x: 0, y: 8
x: 0, y: 16
x: 0, y: 32
x: 0, y: 64
x: 0, y: 128
x: 0, y: 256
x: 0, y: 512


1024

In [7]:
(A 2 4)

x: 2, y: 4
x: 2, y: 3
x: 2, y: 2
x: 2, y: 1
x: 1, y: 2
x: 1, y: 1
x: 0, y: 2
x: 1, y: 4
x: 1, y: 3
x: 1, y: 2
x: 1, y: 1
x: 0, y: 2
x: 0, y: 4
x: 0, y: 8
x: 1, y: 16
x: 1, y: 15
x: 1, y: 14
x: 1, y: 13
x: 1, y: 12
x: 1, y: 11
x: 1, y: 10
x: 1, y: 9
x: 1, y: 8
x: 1, y: 7
x: 1, y: 6
x: 1, y: 5
x: 1, y: 4
x: 1, y: 3
x: 1, y: 2
x: 1, y: 1
x: 0, y: 2
x: 0, y: 4
x: 0, y: 8
x: 0, y: 16
x: 0, y: 32
x: 0, y: 64
x: 0, y: 128
x: 0, y: 256
x: 0, y: 512
x: 0, y: 1024
x: 0, y: 2048
x: 0, y: 4096
x: 0, y: 8192
x: 0, y: 16384
x: 0, y: 32768


65536

In [8]:
(A 3 3)

x: 3, y: 3
x: 3, y: 2
x: 3, y: 1
x: 2, y: 2
x: 2, y: 1
x: 1, y: 2
x: 1, y: 1
x: 0, y: 2
x: 2, y: 4
x: 2, y: 3
x: 2, y: 2
x: 2, y: 1
x: 1, y: 2
x: 1, y: 1
x: 0, y: 2
x: 1, y: 4
x: 1, y: 3
x: 1, y: 2
x: 1, y: 1
x: 0, y: 2
x: 0, y: 4
x: 0, y: 8
x: 1, y: 16
x: 1, y: 15
x: 1, y: 14
x: 1, y: 13
x: 1, y: 12
x: 1, y: 11
x: 1, y: 10
x: 1, y: 9
x: 1, y: 8
x: 1, y: 7
x: 1, y: 6
x: 1, y: 5
x: 1, y: 4
x: 1, y: 3
x: 1, y: 2
x: 1, y: 1
x: 0, y: 2
x: 0, y: 4
x: 0, y: 8
x: 0, y: 16
x: 0, y: 32
x: 0, y: 64
x: 0, y: 128
x: 0, y: 256
x: 0, y: 512
x: 0, y: 1024
x: 0, y: 2048
x: 0, y: 4096
x: 0, y: 8192
x: 0, y: 16384
x: 0, y: 32768


65536

In [9]:
(define (f n) (A 0 n))

In [12]:
(f 10)

x: 0, y: 10


20

In [13]:
(f 20)

x: 0, y: 20


40

`(define (f n) (A 0 n))` computes `n * 2`

In [14]:
(define (g n) (A 1 n))

```s
(g 3)
(A 1 3)
(A (- 1 1) (A 1 (- 3 1))) -> (A 0 (A 1 2))
(A 0 (A (- 1 1) (A 1 (- 2 1)))) -> (A 0 (A 0 (A 1 1)))
(A 0 (A 0 2)) // (A 1 1) -> cond y == 1 -> 2
(A 0 4) // (A 0 2) cond x == 0 -> 2 * 2
8
```

`(g n)` computes `2 * n` by this process repeats `(* 2 y)` N times.

In [4]:
(define (h n) (A 2 n))

```s
(h 3)
(A 2 3)
(A (- 2 1) (A 2 (- 3 1))) -> (A 1 (A 2 2))
(A 1 (A (- 2 1) (A 2 (-2 1)))) -> (A 1 (A 1 (A 2 1))) // <- y == 1
(A 1 (A 1 2))
(A 1 (A (- 1 1) (A 1 (- 2 1)))) -> (A 1 (A 0 (A 1 1))) // <- y == 1
(A 1 (A 0 2)) // <- x == 0
(A 1 4)
(A (- 1 1) (A 1 (- 4 1))) -> (A 0 (A 1 3))
(A 0 (A (- 1 1) (A 1 (- 3 1)))) -> (A 0 (A 0 (A 1 2)))
(A 0 (A 0 (A (- 1 1) (A 1 (- 2 1))))) -> (A 0 (A 0 (A 0 (A 1 1))))  // <- y == 1
(A 0 (A 0 (A 0 2))) // <- x == 0
(A 0 (A 0 4))
(A 0 8)
16
```

`(h n)` computes `2 * n` brcause, after process exanded, this process calcurates y * 2 for n times. 

In [7]:
(h 3)

x: 2, y: 3
x: 2, y: 2
x: 2, y: 1
x: 1, y: 2
x: 1, y: 1
x: 0, y: 2
x: 1, y: 4
x: 1, y: 3
x: 1, y: 2
x: 1, y: 1
x: 0, y: 2
x: 0, y: 4
x: 0, y: 8


16

### 1.2.2 Tree recursion

**plain recursive procedure**
Consider this plain implementation

```s
(define (fib n)
  (cond ((= n 0) 0)
        ((= n 1) 1)
        (else (+ (fib (- n 1)
                 (fib (- n 2)))))))
```

- `(fib 5)` requires `(fib 4)` and `(fib 3)`
- `(fib 4)` requires `(fib 3)` and `(fib 2)`
- ...

**Problems**

fib procedre calls itself twice.

need to compute (fib 1) or (fib 0) precisely Fib(n + 1 ) times

Order grows exponentially with N

**Pros**

Use space linier to N. Be affected by depth of tree.

Simple implementation. Almost same as definition of Fibonacci sequence.

**Iterative procedure**


**Problems**

Need some considerence comareing to the preceding recursive procedure.

**Pros**

This implementation requires steps in linier to N


In [7]:
(define (fib n)
  (define (iter a b count)
    (if (= count 0)
        b
        (iter (+ a b) a (- count 1))))
  (iter 1 0 n))

In [8]:
(fib 5)

5

In [9]:
(fib 6)

8

#### Exmple: Counting change

The number of ways to change amount a using n kinds of coins equals
- the number of ways to change amount a using all but the first kind of coin, plus
- the number of ways to change amount a − d using all n kinds of coins, where d is the denomination of the first kind of coin.

Kind of conins
- $1
- $5
- $10
- $25
- $50

If we need to change amount with $1 and $5 coins, methods can be gouped into
- Do not using $5 coins
- Using $5 coins (= Number of ways to make change for remaining amount)

In procedure,
- Use $5 coins
  - Remain = 0: add 1 method
  - Remain != 0:
    - Use $1 coins
      - Remain = 0: add 1 method
      - Remain != 0: add 0 method

If with $1, $5 and $10 coins
- Use $10 coins
  - Remain = 0: add 1 method
  - Remain != 0:
    - Use $5 coins
      - Remain = 0: add 1 method
      - Remain != 0:
        - Use $1 coins
          - Remain = 0: add 1 method
          - Remain != 0: add 0 method


In [2]:
(define (dollar n)
  (cond
    ((= n 1) 1)
    ((= n 2) 5)
    ((= n 3) 10)
    ((= n 4) 25)
    ((= n 5) 50)))

(define (cc amount kind)
  (display (format "amount: ~a, kind: ~a\n" amount kind))
  (cond
    ((= amount 0) 1)
    ((or (< amount 0) (= kind 0)) 0)
    (else (+ 
            (cc amount (- kind 1))
            (cc (- amount (dollar kind)) kind)
                 ))))

(define (count-change amount) (cc amount 5))

In [3]:
(count-change 10)

amount: 10, kind: 5
amount: 10, kind: 4
amount: 10, kind: 3
amount: 10, kind: 2
amount: 10, kind: 1
amount: 10, kind: 0
amount: 9, kind: 1
amount: 9, kind: 0
amount: 8, kind: 1
amount: 8, kind: 0
amount: 7, kind: 1
amount: 7, kind: 0
amount: 6, kind: 1
amount: 6, kind: 0
amount: 5, kind: 1
amount: 5, kind: 0
amount: 4, kind: 1
amount: 4, kind: 0
amount: 3, kind: 1
amount: 3, kind: 0
amount: 2, kind: 1
amount: 2, kind: 0
amount: 1, kind: 1
amount: 1, kind: 0
amount: 0, kind: 1
amount: 5, kind: 2
amount: 5, kind: 1
amount: 5, kind: 0
amount: 4, kind: 1
amount: 4, kind: 0
amount: 3, kind: 1
amount: 3, kind: 0
amount: 2, kind: 1
amount: 2, kind: 0
amount: 1, kind: 1
amount: 1, kind: 0
amount: 0, kind: 1
amount: 0, kind: 2
amount: 0, kind: 3
amount: -15, kind: 4
amount: -40, kind: 5


4

#### Exercise 1.11

Write a procedure that computes f by means of a recursive process. Write a procedure that computes f by means of an iterative process.

```
          n if n < 3
f (n) = 
          f(n - 1) + 2f(n - 2) + 3f(n - 3) if n >= 3
```

In [34]:
(define (recf n) 
  (define (f n)
    (if (< n 3) 
        n
        (+ (f (- n 1)) (+ (* 2 (f (- n 2)))) (* 3 (f (- n 3))))))
  (f n))

In [35]:
(recf 4)

11

In [12]:
(recf 5)

25

In [47]:
(define (iterf n)
  (define (f a b c n)
    (display (format "~a => a: ~a, b: ~a, c: ~a\n" n a b c))
    (cond 
      ((< n 1) c)
      ((= n 1) b)
      ((= n 2) a)
      (else (f (+ a (* 2 b) (* 3 c)) a b (- n 1)))))
  (f 2 1 0 n))

In [46]:
(iterf 4)

4 => a: 2, b: 1, c: 0
3 => a: 4, b: 2, c: 1
2 => a: 11, b: 4, c: 2


11

In [44]:
(iterf 5)

5 => a: 2, b: 1, c: 0
4 => a: 4, b: 2, c: 1
3 => a: 11, b: 4, c: 2
2 => a: 25, b: 11, c: 4


25

In [54]:
(iterf 2)

2 => a: 2, b: 1, c: 0


2

#### Exercise 1.12: Pascal’s triangle

```
    1
   1 1 
  1 2 1 
 1 3 3 1 
1 4 6 4 1
   ...
```
The numbers at the edge of the triangle are all 1, and each number inside the triangle is the sum of the two numbers above it. Write a procedure that computes element of Pascal’s triangle by means of a recursive process.

In [15]:
(define (triangle n k)
  (if (or (= n k) (= k 1))
      1
      (+ (triangle (- n 1) k)
         (triangle (- n 1) (- k 1)))))

In [19]:
(display (format "~a\n" (triangle 7 1)))
(display (format "~a\n" (triangle 7 2)))
(display (format "~a\n" (triangle 7 3)))
(display (format "~a\n" (triangle 7 4)))
(display (format "~a\n" (triangle 7 5)))
(display (format "~a\n" (triangle 7 6)))
(display (format "~a\n" (triangle 7 7)))

1
6
15
20
15
6
1


### 1.2.3 Orders of Growth


### 1.2.4 Exponentiaion


In [27]:
(define (exp-rec b n)
  (define (exp b n)
    (if (= n 1)
        b
        (* b (exp b (- n 1)))))
  (exp b n))

In [28]:
(display (format "~a\n" (exp-rec 2 1)))
(display (format "~a\n" (exp-rec 2 2)))
(display (format "~a\n" (exp-rec 2 3)))
(display (format "~a\n" (exp-rec 2 4)))
(display (format "~a\n" (exp-rec 2 5)))

2
4
8
16
32


In [33]:
(define (exp-iter b n)
  (define (exp b n acc)
    (if (= n 1)
        acc
        (exp b (- n 1) (* acc b))))
  (exp b n b))

In [34]:
(display (format "~a\n" (exp-iter 2 1)))
(display (format "~a\n" (exp-iter 2 2)))
(display (format "~a\n" (exp-iter 2 3)))
(display (format "~a\n" (exp-iter 2 4)))
(display (format "~a\n" (exp-iter 2 5)))

2
4
8
16
32


This recuresive process requires O(n) steps and O(n) space.

The iterative version requires O(n) strps and O(1) space.

We can reduce steps by 
```
b ^ n = (b ^ n/2) ^ 2 if n % 2 == 0
b ^ n = b * b ^ (n - 1) if n % 2 == 1
```

In [43]:
(define (fast-exp b n)
  (define (square n) (* n n))
  (define (exp b n)
    (cond 
      ((= n 0) 1)
      ((even? n) (square (exp b (/ n 2))))
      (else (* b (exp b (- n 1))))))
  (exp b n))

In [44]:
(display (format "~a\n" (fast-exp 2 1)))
(display (format "~a\n" (fast-exp 2 2)))
(display (format "~a\n" (fast-exp 2 3)))
(display (format "~a\n" (fast-exp 2 4)))
(display (format "~a\n" (fast-exp 2 5)))

2
4
8
16
32


#### Exercise 1.16: 
Design a procedure that evolves an iterative exponentiation process that uses successive squaring and uses a logarithmic number of steps, as does fast-expt.
Hint: Using the observation that (b ^ n/2) ^ 2 = (b ^ 2) ^ n/2

In [32]:
(define (fast-exp-iter b n)
  (define (square n) (* n n))
  (define (exp b n acc)
    (cond
      ((= n 1) acc)
      ((even? n) (exp b (/ n 2) (* (square b) acc)))
      (else (exp b (- n 1) (* acc b)))))
  (exp b n 1))

In [65]:
(display (format "~a\n" (fast-exp-iter 2 1)))
(display (format "~a\n" (fast-exp-iter 2 2)))
(display (format "~a\n" (fast-exp-iter 2 3)))
(display (format "~a\n" (fast-exp-iter 2 4)))
(display (format "~a\n" (fast-exp-iter 2 5)))

1
4
8
16
32


#### Exercise 1.17
```s
(define (* a b)
  (if (= b 0)
      0
      (+ a (* a (- b 1)))))
```

Now suppose we include, together with 
- addition
- operations `double`, which doubles an integer, 
- `halve`, which divides an (even) integer by 2.
Using these, design a multiplication procedure analogous to `fast-expt` that uses a logarithmic number of steps.

In [112]:
(define (mul a b)
  (define (double n) (* n 2))
  (define (halve n) (/ n 2))
  (cond
    ((= b 0) 1)
    ((= b 1) a)
    ((even? b) (double (mul a (halve b))))
    (else (+ a (mul a (- b 1))))))

In [113]:
(mul 2 0)

1

In [116]:
(display (format "~a\n" (mul 2 0)))
(display (format "~a\n" (mul 6 1)))
(display (format "~a\n" (mul 2 2)))
(display (format "~a\n" (mul 2 10)))
(display (format "~a\n" (mul 3 4)))
(display (format "~a\n" (mul 5 5)))
(display (format "~a\n" (mul 8 6)))

1
6
4
20
12
25
48


#### Exercise 1.18
Using the results of Exercise 1.16 and Exercise 1.17, devise a procedure that generates an iterative process for multiplying two integers in terms of adding, doubling, and halving and uses a logarithmic number of steps.

In [17]:
(define (mul-iter a b)
  (define (double n) (* n 2))
  (define (halve n) (/ n 2))
  (define (mul a b acc)
    ;;(display (format "a: ~a, b: ~a, acc: ~a\n" a b acc))
    (cond
      ((= b 0) acc)
      ((even? b) (mul (double a) (halve b) acc))
      (else (mul a (- b 1) (+ acc a)))
      ))
  (mul a b 0))

In [16]:
(mul-iter 2 10)

a: 2, b: 10, acc: 0
a: 4, b: 5, acc: 0
a: 4, b: 4, acc: 4
a: 8, b: 2, acc: 4
a: 16, b: 1, acc: 4
a: 16, b: 0, acc: 20


20

In [14]:
(display (format "~a\n" (mul-iter 6 1)))
(display (format "~a\n" (mul-iter 2 2)))
(display (format "~a\n" (mul-iter 2 10)))
(display (format "~a\n" (mul-iter 3 4)))
(display (format "~a\n" (mul-iter 5 5)))
(display (format "~a\n" (mul-iter 8 6)))

6
4
20
12
25
48


#### Exercise 1.19: a clever algorithm for computing the Fibonacci

T: Transformation to bind a <- a + b, b <- a

T^n: the N-th Fibonacci number

Tpq, p = 0, q = 1
a <- bq + aq, b <- bp + aq

- Show these operations have same efflect.
  - apply `Tpq` twice
  - apply `Tp'q'` once
- Compute the values of `p'` and `q'`.

p' = p^2 + q^2
q' = q^2 + 2pq

This gives us an explicit way to square these transformations, and thus we can compute T n using successive squaring, as in the fast-expt procedure

```s
(define (fib n) 
  (define (fib-iter a b p q count) 
    (cond ((= count 0) b)
          ((even? count) (fib-iter a
                                   b
                                   ⟨??⟩ ; compute p′
                                   ⟨??⟩ ; compute q′ 
                                   (/ count 2)))
          (else (fib-iter (+ (* b q) (* a q) (* a p))
                          (+ (* b p) (* a q))
                          p
                          q
                          (- count 1)))))
  (fib-iter 1 0 0 1 n))
  ```

In [33]:
(define (logarithmic-fib n) 
  (define (fib-iter a b p q count) 
    (cond ((= count 0) b)
          ((even? count) (fib-iter a
                                   b
                                   (+ (* p p) (* q q)) ;; p' = p^2 + q^2
                                   (+ (* q q) (* 2 p q));; q' = q^2 + 2pq
                                   (/ count 2)))
          (else (fib-iter (+ (* b q) (* a q) (* a p))
                          (+ (* b p) (* a q))
                          p
                          q
                          (- count 1)))))
  (fib-iter 1 0 0 1 n))

In [39]:
(display (format "~a\n" (logarithmic-fib 1)))
(display (format "~a\n" (logarithmic-fib 2)))
(display (format "~a\n" (logarithmic-fib 3)))
(display (format "~a\n" (logarithmic-fib 4)))
(display (format "~a\n" (logarithmic-fib 5)))
(display (format "~a\n" (logarithmic-fib 6)))
(display (format "~a\n" (logarithmic-fib 7)))
(display (format "~a\n" (logarithmic-fib 8)))
(display (format "~a\n" (logarithmic-fib 9)))

1
1
2
3
5
8
13
21
34


### 1.2.5 Greatest Common Divisors

- `r` is the remainder of `a / b`
- common divisors of `a` and `b`, are same as ones of `b` and `r`
- GCD(a, b) = GCD(b, r)

```
GCD(206, 40) 
= GCD(40, 6)
= GCD(6, 4)
= GCD(4, 2)
= GCD(2, 0)
= 2
```


In [3]:
(define (gcd a b)
  (if (= b 0)
    a
    (gcd b (remainder a b))))

In [4]:
(gcd 206 40)

2

### 1.2.6 Example: Testing for Primality

Introduce two methods
- 1: Order of grouth O(√n)
- 2: Order of grouth O(log n), "probabilistic" algorithm

#### 1st method

In [51]:
(define (square n) (* n n))
(define (divides? a b) (= (remainder b a) 0))

(define (smallest-divisor n)
  (define (find-divisor n test)
    (cond
      ((> (square test) n) n)
      ((divides? test n) test)
      (else (find-divisor n (+ test 1)))))
  (find-divisor n 2))

In [14]:
(display (format "~a\n" (smallest-divisor 1)))
(display (format "~a\n" (smallest-divisor 2)))
(display (format "~a\n" (smallest-divisor 7)))
(display (format "~a\n" (smallest-divisor 9)))
(display (format "~a\n" (smallest-divisor 21)))
(display (format "~a\n" (smallest-divisor 37)))

1
2
7
3
3
37


In [52]:
(define (prime? n)
  (= n (smallest-divisor n)))

In [20]:
(display (format "~a\n" (prime? 1)))
(display (format "~a\n" (prime? 2)))
(display (format "~a\n" (prime? 7)))
(display (format "~a\n" (prime? 9)))
(display (format "~a\n" (prime? 21)))
(display (format "~a\n" (prime? 37)))

True
True
True
False
False
True


This process based on the fact that if n is not prime, it must have a divisor less than or equal to `√n`.

-> It requires O(√n) step because it iterates `test` value 2 to √n

#### 2nd method: The Fermat test

Fermat’s Little Theorem
```md
If `n` is a prime number and `a` is any positive integer less than `n`, then `a` raised to the nth power is congruent to `a` modulo `n`.
```

Two numbers are said to be congruent modulo `n` if they both have the same remainder when divided by `n`. The remainder of a number `a` when divided by `n` is also referred to as the `remainder of a modulo n`, or simply as a `modulo n`

- Given
  - n: target
  - a: 0 < a < n
- When
  - a ^ n mod n = a
- Then
  - n is a prime number


In [40]:
(define (expmod base exp m)
  (define (square n) (* n n))
  ; (display (format "base: ~a, exp: ~a, m: ~a\n" base exp m))
  (cond ((= exp 0) 1)
        ((even? exp)
          (remainder
            (square (expmod base (/ exp 2) m))
            m))
        (else
          (remainder
            (* base (expmod base (- exp 1) m))
            m))))

In [37]:
(expmod 4 2 1)

base: 4, exp: 2, m: 1
base: 4, exp: 1, m: 1
base: 4, exp: 0, m: 1


0

In [34]:
(define (fermat-test n)
  (define (try-it a)
    ; (display (format "a: ~a, expmod: ~a\n" a (expmod a n n)))
    (= (expmod a n n) a))
  (try-it (+ 1 (random (- n 1)))))

In [52]:
(fermat-test 7)

base: 3, exp: 7, m: 7
base: 3, exp: 6, m: 7
base: 3, exp: 3, m: 7
base: 3, exp: 2, m: 7
base: 3, exp: 1, m: 7
base: 3, exp: 0, m: 7


#t

In [42]:
(fermat-test 7)

base: 4, exp: 7, m: 7
base: 4, exp: 6, m: 7
base: 4, exp: 3, m: 7
base: 4, exp: 2, m: 7
base: 4, exp: 1, m: 7
base: 4, exp: 0, m: 7


#t

In [41]:
(fermat-test 57)

base: 22, exp: 57, m: 57
base: 22, exp: 56, m: 57
base: 22, exp: 28, m: 57
base: 22, exp: 14, m: 57
base: 22, exp: 7, m: 57
base: 22, exp: 6, m: 57
base: 22, exp: 3, m: 57
base: 22, exp: 2, m: 57
base: 22, exp: 1, m: 57
base: 22, exp: 0, m: 57


#f

#### Probabilistic methods

Fermat's little theorem is not a quire correct assertion because there are numbers thar fool it.

number `n` have these property fool the throrem, but such kind of numbers are rare.
- not prime
- `a^n` is congrument to `a mod n` for all integers `a < n`


#### Exercise 1.21
Use the smallest-divisor procedure to find the smallest divisor of each of the following numbers: 199, 1999, 19999.

In [56]:
(smallest-divisor 199)

199

In [57]:
(smallest-divisor 1999)

1999

In [58]:
(smallest-divisor 19999)

7

#### Exercise 1.22


In [6]:
(current-time)

1644886693.782627

In [49]:
(define (runtime) (current-time))

In [46]:
(define (timed-prime-test n) 
  (newline)
  (display n)
  (start-prime-test n (runtime)))
(define (start-prime-test n start-time)
  (if (prime? n)
    (report-prime (- (runtime) start-time))))
(define (report-prime elapsed-time)
  (display " *** ")
  (display (* elapsed-time 1000)))


Using this procedure, write a procedure `search-for-primes` that checks the primality of consecutive odd integers in a specified range.

In [64]:
(timed-prime-test 7)


7 *** 2.3543834686279297

In [16]:
(timed-prime-test 4) 


4

#f

In [47]:
(define (search-for-primes min max)
  (define (iter n)
    (cond ((<= n max)
            (timed-prime-test n)
            (iter (+ n 2)))))
  (if (odd? min)
        (iter min)
        (iter (+ min 1))))

In [65]:
(search-for-primes 10 20)


11 *** 3.9298534393310547
13 *** 5.522012710571289
15
17 *** 3.8318634033203125
19 *** 4.277944564819336

#f

In [67]:
(search-for-primes 1000 1100)


1001
1003
1005
1007
1009 *** 29.5259952545166
1011
1013 *** 29.17933464050293
1015
1017
1019 *** 30.757904052734375
1021 *** 27.501821517944336
1023
1025
1027
1029
1031 *** 27.013063430786133
1033 *** 28.365135192871094
1035
1037
1039 *** 28.81908416748047
1041
1043
1045
1047
1049 *** 31.595945358276367
1051 *** 28.053998947143555
1053
1055
1057
1059
1061 *** 32.98687934875488
1063 *** 29.25896644592285
1065
1067
1069 *** 30.61389923095703
1071
1073
1075
1077
1079
1081
1083
1085
1087 *** 26.378870010375977
1089
1091 *** 26.710033416748047
1093 *** 27.255773544311523
1095
1097 *** 27.603864669799805
1099

#f

In [68]:
(search-for-primes 10000 10100)


10001
10003
10005
10007 *** 86.4858627319336
10009 *** 99.11918640136719
10011
10013
10015
10017
10019
10021
10023
10025
10027
10029
10031
10033
10035
10037 *** 105.9882640838623
10039 *** 105.92007637023926
10041
10043
10045
10047
10049
10051
10053
10055
10057
10059
10061 *** 110.29767990112305
10063
10065
10067 *** 84.96713638305664
10069 *** 75.65784454345703
10071
10073
10075
10077
10079 *** 108.1550121307373
10081
10083
10085
10087
10089
10091 *** 119.28606033325195
10093 *** 255.9349536895752
10095
10097
10099 *** 67.72708892822266

#f

In [69]:
(search-for-primes 100000 100100)


100001
100003 *** 322.96204566955566
100005
100007
100009
100011
100013
100015
100017
100019 *** 410.10403633117676
100021
100023
100025
100027
100029
100031
100033
100035
100037
100039
100041
100043 *** 238.18397521972656
100045
100047
100049 *** 253.7992000579834
100051
100053
100055
100057 *** 243.60203742980957
100059
100061
100063
100065
100067
100069 *** 295.4108715057373
100071
100073
100075
100077
100079
100081
100083
100085
100087
100089
100091
100093
100095
100097
100099

#f

In [15]:
(define (av-3 a b c) (/ (+ a b c) 3))

In [72]:
(av-3
  29.5259952545166
  29.17933464050293
  30.757904052734375)

29.821077982584637

In [73]:
(av-3
  86.4858627319336
  99.11918640136719
  105.9882640838623)

97.1977710723877

In [74]:
(av-3
  322.96204566955566
  410.10403633117676
  238.18397521972656)

323.7500190734863

`(search-for-primes 1000 1100)`
```
1009 *** 29.5259952545166
1013 *** 29.17933464050293
1019 *** 30.757904052734375
```
average: 29.821077982584637

`(search-for-primes 10000 10100)`
```
10007 *** 86.4858627319336
10009 *** 99.11918640136719
10037 *** 105.9882640838623
```
average: 97.1977710723877

`(search-for-primes 100000 100100)`
```
100003 *** 322.96204566955566
100019 *** 410.10403633117676
100043 *** 238.18397521972656
```
average: 323.7500190734863

If the number to test increase `* 10`, this procedure takes times almost `* √10 (≒ 3.16227766017)` longer. Therefore we may say this procedure has order of growth of O(√n).

```s
(/ 97.1977710723877 29.821077982584637) 
; => 3.259364773102794

(/ 323.7500190734863 97.1977710723877) 
;=> 3.3308378937246887
```

#### Exercise 1.23

```s
(define (smallest-divisor n)
  (define (find-divisor n test)
    (cond
      ((> (square test) n) n)
      ((divides? test n) test)
      (else (find-divisor n (+ test 1)))))
  (find-divisor n 2))
```

This implementation is redundant. We don't need to check even number except 2.

Define a procedure `next` that returns ...
- 3 if its input is 2
- input plus 2 if input is even except 2

1. implement prime test with `next` procedure
2. compare time with previous version 


In [6]:
(define (smallest-divisor2 n)
  (define (square n) (* n n))
  (define (divides? a b) (= (remainder b a) 0))
  (define (next n)
    (if (= n 2) 3
                (+ n 2)))
  (define (find-divisor n test)
    (cond
      ((> (square test) n) n)
      ((divides? test n) test)
      (else (find-divisor n (next test)))))
  (find-divisor n 2))

In [7]:
(display (format "~a\n" (smallest-divisor2 1)))
(display (format "~a\n" (smallest-divisor2 2)))
(display (format "~a\n" (smallest-divisor2 7)))
(display (format "~a\n" (smallest-divisor2 9)))
(display (format "~a\n" (smallest-divisor2 21)))
(display (format "~a\n" (smallest-divisor2 37)))

1
2
7
3
3
37


In [11]:
(define (timed-prime-test2 n)
  (define (prime? n)
    (= n (smallest-divisor2 n)))
  (define (runtime) (current-time))
  (define (start-prime-test n start-time)
    (if (prime? n)
      (report-prime (- (runtime) start-time))))
  (define (report-prime elapsed-time)
    (display " *** ")
    (display (* elapsed-time 1000)))

  (newline)
  (display n)
  (start-prime-test n (runtime)))


In [22]:
(display (format "~a" (timed-prime-test2 1009)))
(display (format "~a" (timed-prime-test2 1013)))
(display (format "~a" (timed-prime-test2 1019)))
(display (format "~a" (timed-prime-test2 10007)))
(display (format "~a" (timed-prime-test2 10009)))
(display (format "~a" (timed-prime-test2 10037)))
(display (format "~a" (timed-prime-test2 100003)))
(display (format "~a" (timed-prime-test2 100019)))
(display (format "~a" (timed-prime-test2 100043)))


1009 *** 16.996145248413086<void>
1013 *** 16.336917877197266<void>
1019 *** 16.31617546081543<void>
10007 *** 46.724796295166016<void>
10009 *** 43.47085952758789<void>
10037 *** 44.152021408081055<void>
100003 *** 147.1848487854004<void>
100019 *** 142.9271697998047<void>
100043 *** 150.0070095062256<void>

In [23]:
(display (format "1009: ~a\n" (/ 16.996145248413086 29.5259952545166)))
(display (format "1013: ~a\n" (/ 16.336917877197266 29.17933464050293)))
(display (format "1019: ~a\n" (/ 16.31617546081543 29.17933464050293)))

(display (format "10007: ~a\n" (/ 46.724796295166016 86.4858627319336)))
(display (format "10009: ~a\n" (/ 43.47085952758789 99.11918640136719)))
(display (format "10037: ~a\n" (/ 44.152021408081055 105.9882640838623)))

(display (format "100003: ~a\n" (/ 147.1848487854004 322.96204566955566)))
(display (format "100019: ~a\n" (/ 142.9271697998047 410.10403633117676)))
(display (format "100043: ~a\n" (/ 150.0070095062256 238.18397521972656)))


1009: 0.5756332716951575
1013: 0.5598797257878696
1019: 0.5591688659743274
10007: 0.5402593536008469
10009: 0.4385715935112668
10037: 0.41657462540518775
100003: 0.45573419774531393
100019: 0.3485144186300688
100043: 0.6297947180025145


Now prime test procedure takes almost halv time compare to previous version. Improvement ratio becomes larger as the number becomes larger. I guess it is due to setup time of procedures. The cost of initialization occupies less as the whole number of steps become larger.

#### Exercise 1.24

In [53]:
(search-for-primes 1000000 1000050)


1000001
1000003 *** 652.545690536499
1000005
1000007
1000009
1000011
1000013
1000015
1000017
1000019
1000021
1000023
1000025
1000027
1000029
1000031
1000033 *** 779.0577411651611
1000035
1000037 *** 726.5751361846924
1000039 *** 803.3170700073242
1000041
1000043
1000045
1000047
1000049

#f

In [41]:
(define (timed-prime-test3 n)
  (define (runtime) (current-time))
  (define (start-prime-test n start-time)
    (if (fermat-test n)
      (report-prime (- (runtime) start-time))))
  (define (report-prime elapsed-time)
    (display " *** ")
    (display (* elapsed-time 1000)))

  (newline)
  (display n)
  (start-prime-test n (runtime)))


In [55]:
(display (format "~a" (timed-prime-test3 1009)))
(display (format "~a" (timed-prime-test3 1013)))
(display (format "~a" (timed-prime-test3 1019)))
(display (format "~a" (timed-prime-test3 1000003)))
(display (format "~a" (timed-prime-test3 1000033)))
(display (format "~a" (timed-prime-test3 1000037)))


1009 *** 14.216899871826172<void>
1013 *** 15.447139739990234<void>
1019 *** 15.090227127075195<void>
1000003 *** 21.270036697387695<void>
1000033 *** 23.38695526123047<void>
1000037 *** 23.550987243652344<void>

In [56]:
(display (format "~a\n" (av-3 14.216899871826172 15.447139739990234 15.090227127075195)))
(display (format "~a\n" (av-3 21.270036697387695 23.38695526123047 23.550987243652344)))

14.918088912963867
22.735993067423504


- log 1000 (= 3) 
- log 1000000 (= 6)

In O(n) description, Scheme needs double steps to calcurate primes near 1,000,000 compare to primes near 1,000.
In actual time, it took approximately 1.5 times longer to for primes near 1,000,000.

#### Exercise 1.25

No. Scheme needs amount of memory to calcurate `(fast-exp base exp)`.